In [1]:
import pandas as pd
import arff as liacarff

# 加载 ARFF 文件
with open('PROMISE_exp.arff', 'r') as f:
    data_dict = liacarff.load(f)

# 提取数据
data = data_dict['data']

# 转换为 DataFrame
df = pd.DataFrame(data, columns=[attr[0] for attr in data_dict['attributes']])
df['ProjectID'] = df['ProjectID'].astype(int)  # 将 ProjectID 列转换为整数类型

# 显示 DataFrame
print(df)

     ProjectID                                    RequirementText _class_
0            1  The system shall refresh the display every 60 ...      PE
1            1  The application shall match the color of the s...      LF
2            1  If projected  the data must be readable.  On a...      US
3            1  The product shall be available during normal b...       A
4            1  If projected  the data must be understandable....      US
..         ...                                                ...     ...
965         48  Registered User must be able to maintain his/h...       F
966         48  The entire website must be user-friendly and e...      US
967         48  The system shall support up to 10000 simultane...      PE
968         48  The website must provide highest degree of sec...      SE
969         49  The software application should be easily tran...      PO

[970 rows x 3 columns]


In [2]:
# DATA PREPROCESSING
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import string

# 下载NLTK的停用词和词性标注器数据
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# lower
df['RequirementText'] = df['RequirementText'].str.lower()

# Remove punctuation, leading and trailing spaces
df['RequirementText'] = df['RequirementText'].apply(lambda x: re.sub(f"[{re.escape(string.punctuation)}]", "", x.strip()))

# 分词
df['RequirementText'] = df['RequirementText'].apply(word_tokenize)

# stopwords
stop_words = set(stopwords.words('english'))
df['RequirementText'] = df['RequirementText'].apply(lambda x: [word for word in x if word not in stop_words])

# stemming
# stemmer = PorterStemmer()
# df['RequirementText'] = df['RequirementText'].apply(lambda x: [stemmer.stem(word) for word in x])

# lemmatization
lemmatizer = WordNetLemmatizer()
df['RequirementText'] = df['RequirementText'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# 合并词语为字符串
df['RequirementText'] = df['RequirementText'].apply(lambda x: ' '.join(x))

# Replace multiple spaces with a single space
df['RequirementText'] = df['RequirementText'].apply(lambda x: re.sub(' +', ' ', x))

# 移除整个列中每个字符串首尾的全部空格
df['RequirementText'] = df['RequirementText'].str.strip()

# 查看处理后的数据集
print(df.head())

[nltk_data] Downloading package stopwords to /home/li/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/li/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/li/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


   ProjectID                                    RequirementText _class_
0          1       system shall refresh display every 60 second      PE
1          1  application shall match color schema set forth...      LF
2          1  projected data must readable 10x10 projection ...      US
3          1  product shall available normal business hour l...       A
4          1  projected data must understandable 10x10 proje...      US


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [4]:
class_distribution = df['_class_'].value_counts()
print(class_distribution)

_class_
F     444
SE    125
US     85
O      77
PE     67
LF     49
A      31
MN     24
SC     22
FT     18
L      15
PO     13
Name: count, dtype: int64


In [5]:
from sklearn.model_selection import train_test_split
y = df['_class_']
X = df['RequirementText']

# 假设 X 是特征，y 是目标变量（类别标签）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

print(y_test)
class_distribution = y_test.value_counts()
print(class_distribution)

222    SC
708     F
362     F
67      F
206     O
       ..
59      F
639     F
452     F
225     O
775    MN
Name: _class_, Length: 97, dtype: object
_class_
F     44
SE    13
US     8
O      8
PE     7
LF     5
A      3
SC     2
L      2
MN     2
FT     2
PO     1
Name: count, dtype: int64


In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 初始化SVM模型
svm_model = SVC(kernel='linear', C=1.0)

# 训练模型
svm_model.fit(X_train_tfidf, y_train)

# 在测试集上进行预测
y_pred_svm = svm_model.predict(X_test_tfidf)

# 评估模型性能
print("Accuracy (SVM):", accuracy_score(y_test, y_pred_svm))
print("Classification Report (SVM):\n", classification_report(y_test, y_pred_svm))

Accuracy (SVM): 0.6701030927835051
Classification Report (SVM):
               precision    recall  f1-score   support

           A       0.67      0.67      0.67         3
           F       0.70      0.95      0.81        44
          FT       0.00      0.00      0.00         2
           L       0.50      0.50      0.50         2
          LF       0.40      0.40      0.40         5
          MN       0.50      0.50      0.50         2
           O       0.67      0.50      0.57         8
          PE       1.00      0.57      0.73         7
          PO       0.00      0.00      0.00         1
          SC       0.00      0.00      0.00         2
          SE       0.67      0.46      0.55        13
          US       0.60      0.38      0.46         8

    accuracy                           0.67        97
   macro avg       0.48      0.41      0.43        97
weighted avg       0.65      0.67      0.64        97



/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# 定义Logistic Regression模型
lr_model = LogisticRegression(solver='liblinear')

# 定义要调优的超参数网格
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}

# 创建GridSearchCV对象
grid_search = GridSearchCV(lr_model, param_grid, cv=5, scoring='f1_weighted')

# 执行Grid Search以选择最佳超参数配置
grid_search.fit(X_train_tfidf, y_train)

# 输出最佳超参数配置
print("Best hyperparameters: ", grid_search.best_params_)

# 使用最佳超参数配置的模型进行预测
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tfidf)

# 输出在测试集上的准确率
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on test set: {:.2f}%".format(accuracy * 100))
print("Classification Report (KNN):\n", classification_report(y_test, y_pred))

Best hyperparameters:  {'C': 10, 'penalty': 'l2'}
Accuracy on test set: 69.07%
Classification Report (KNN):
               precision    recall  f1-score   support

           A       0.75      1.00      0.86         3
           F       0.75      0.89      0.81        44
          FT       0.50      0.50      0.50         2
           L       0.50      0.50      0.50         2
          LF       0.33      0.20      0.25         5
          MN       0.50      0.50      0.50         2
           O       0.83      0.62      0.71         8
          PE       1.00      0.43      0.60         7
          PO       0.00      0.00      0.00         1
          SC       0.50      0.50      0.50         2
          SE       0.60      0.46      0.52        13
          US       0.55      0.75      0.63         8

    accuracy                           0.69        97
   macro avg       0.57      0.53      0.53        97
weighted avg       0.69      0.69      0.67        97



/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# 定义欠采样器
under_sampler = RandomUnderSampler(random_state=42)

# 对训练集进行欠采样
X_train_resampled, y_train_resampled = under_sampler.fit_resample(X_train_tfidf, y_train)

# 在欠采样后的数据上训练模型（这里使用了SVM作为示例）
svm_model = SVC()
svm_model.fit(X_train_resampled, y_train_resampled)

# 对测试集进行转换
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# 在测试集上进行预测
y_pred = svm_model.predict(X_test_tfidf)

# 评估模型性能
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.30927835051546393
Classification Report:
               precision    recall  f1-score   support

           A       0.50      1.00      0.67         3
           F       0.55      0.14      0.22        44
          FT       0.11      1.00      0.20         2
           L       0.67      1.00      0.80         2
          LF       0.09      0.20      0.13         5
          MN       0.33      0.50      0.40         2
           O       0.56      0.62      0.59         8
          PE       1.00      0.14      0.25         7
          PO       0.00      0.00      0.00         1
          SC       0.00      0.00      0.00         2
          SE       0.33      0.23      0.27        13
          US       0.26      0.75      0.39         8

    accuracy                           0.31        97
   macro avg       0.37      0.47      0.33        97
weighted avg       0.47      0.31      0.29        97



/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
# F binary

# df['binary_label'] = df['_class_'].apply(lambda x: 1 if x == 'F' else 0)
# X_train, X_test, y_train, y_test = train_test_split(df['RequirementText'], df['binary_label'], test_size=0.2, random_state=42)

# # 使用TfidfVectorizer进行转换
# ngram_range = (1,2)
# min_df = 0.01 # ignore terms that appear in less than 1% of the documents
# max_df = 0.8 # ignore terms that appear in more than 80% of the documents

# tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
#                         ngram_range=ngram_range,
#                         stop_words=None,
#                         lowercase=False,
#                         max_df=max_df,
#                         min_df=min_df,
#                         norm='l2',
#                         sublinear_tf=True)

# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)

# model_tfidf = MultinomialNB()
# model_tfidf.fit(X_train_tfidf, y_train)

# y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

# print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
# print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf))

In [10]:
# filtered_df = df[df['_class_'] != 'F']
# X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# # 使用TfidfVectorizer进行转换
# ngram_range = (1,2)
# min_df = 0.01 # ignore terms that appear in less than 1% of the documents
# max_df = 0.8 # ignore terms that appear in more than 80% of the documents

# tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
#                         ngram_range=ngram_range,
#                         stop_words=None,
#                         lowercase=False,
#                         max_df=max_df,
#                         min_df=min_df,
#                         norm='l2',
#                         sublinear_tf=True)

# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)

# model_tfidf = MultinomialNB()
# model_tfidf.fit(X_train_tfidf, y_train)

# y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

# print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
# print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

In [11]:
filtered_df = df[df['_class_'].isin(['MN', 'SC', 'PE'])]
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.6086956521739131
Classification Report (TF-IDF):
               precision    recall  f1-score   support

          MN       1.00      0.00      0.00         5
          PE       0.61      1.00      0.76        14
          SC       1.00      0.00      0.00         4

    accuracy                           0.61        23
   macro avg       0.87      0.33      0.25        23
weighted avg       0.76      0.61      0.46        23



In [12]:
filtered_df = df[df['_class_'].isin(['A', 'FT'])]
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.7
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           A       0.67      1.00      0.80         6
          FT       1.00      0.25      0.40         4

    accuracy                           0.70        10
   macro avg       0.83      0.62      0.60        10
weighted avg       0.80      0.70      0.64        10



In [13]:
filtered_df = df[df['_class_'].isin(['SE', 'L'])]
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.8928571428571429
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           L       1.00      0.00      0.00         3
          SE       0.89      1.00      0.94        25

    accuracy                           0.89        28
   macro avg       0.95      0.50      0.47        28
weighted avg       0.90      0.89      0.84        28



In [14]:
filtered_df = df[df['_class_'].isin(['O', 'US'])]
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf, zero_division=1))

Accuracy (TF-IDF): 0.8484848484848485
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           O       0.92      0.75      0.83        16
          US       0.80      0.94      0.86        17

    accuracy                           0.85        33
   macro avg       0.86      0.85      0.85        33
weighted avg       0.86      0.85      0.85        33

